In [1]:
#All the header files required for the code
import numpy as np
import pandas as pd
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
import random

In [2]:
#Importing both the file using pandas 
data1 = pd.read_csv('movies data.csv')
data2 = pd.read_csv('ratings data.csv')

In [3]:
#Deleting unnecessary columns
data1 = data1.drop('Unnamed: 0',axis = 1)
data2 = data2.drop(['Unnamed: 0','Timestamp'],axis = 1)

In [4]:
data1.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
data2.head()

,UserID,MovieID,Rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [6]:
#Merging both the dataframes
data = pd.merge(data2 , data1 , how='outer', on='MovieID')

In [7]:
data.head()

,UserID,MovieID,Rating,Title,Genres
0,1.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama
1,2.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama
2,12.0,1193,4.0,One Flew Over the Cuckoo's Nest (1975),Drama
3,15.0,1193,4.0,One Flew Over the Cuckoo's Nest (1975),Drama
4,17.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama


In [8]:
# Data Processing
# Converting Genres into different columns 
# Here we just create columns and put there initial value as 0
x = data.Genres
a = list()
for i in x:
    abc = i
    a.append(abc.split('|'))
a = pd.DataFrame(a)   
b = a[0].unique()
for i in b:
    data[i] = 0
data.head(2)

,UserID,MovieID,Rating,Title,Genres,Drama,Animation,Musical,Action,Comedy,...,Thriller,Crime,Western,Documentary,Mystery,Horror,Sci-Fi,Film-Noir,War,Fantasy
0,1.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# we assign 1 to all the columns which are present in the Genres
for i in b:
    data.loc[data['Genres'].str.contains(i), i] = 1

In [10]:
data.head(2)

,UserID,MovieID,Rating,Title,Genres,Drama,Animation,Musical,Action,Comedy,...,Thriller,Crime,Western,Documentary,Mystery,Horror,Sci-Fi,Film-Noir,War,Fantasy
0,1.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Now there is no use of genre 
# Since we have movie id so there is no need for movie names as well
data = data.drop(['Genres','Title'],axis =1)
data.head()

,UserID,MovieID,Rating,Drama,Animation,Musical,Action,Comedy,Adventure,Romance,...,Thriller,Crime,Western,Documentary,Mystery,Horror,Sci-Fi,Film-Noir,War,Fantasy
0,1.0,1193,5.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.0,1193,5.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12.0,1193,4.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15.0,1193,4.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,17.0,1193,5.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
data.columns

Index(['UserID', 'MovieID', 'Rating', 'Drama', 'Animation', 'Musical',
       'Action', 'Comedy', 'Adventure', 'Romance', 'Children's', 'Thriller',
       'Crime', 'Western', 'Documentary', 'Mystery', 'Horror', 'Sci-Fi',
       'Film-Noir', 'War', 'Fantasy'],
      dtype='object')

In [13]:
# Because of merging some null values are created
data.isnull().sum()

UserID         177
MovieID          0
Rating         177
Drama            0
Animation        0
Musical          0
Action           0
Comedy           0
Adventure        0
Romance          0
Children's       0
Thriller         0
Crime            0
Western          0
Documentary      0
Mystery          0
Horror           0
Sci-Fi           0
Film-Noir        0
War              0
Fantasy          0
dtype: int64

In [14]:
#WE simply drop the null values coz the are not treatable
data.dropna(inplace= True )

In [15]:
data.isnull().sum()

UserID         0
MovieID        0
Rating         0
Drama          0
Animation      0
Musical        0
Action         0
Comedy         0
Adventure      0
Romance        0
Children's     0
Thriller       0
Crime          0
Western        0
Documentary    0
Mystery        0
Horror         0
Sci-Fi         0
Film-Noir      0
War            0
Fantasy        0
dtype: int64

In [16]:
#By different meathods I found 8 cluster are better 
kmeanModel = KMeans(n_clusters=8)
kmeanModel.fit(data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [17]:
# Creating an extra column in data for storing the cluster values
data['Cluster'] = kmeanModel.labels_
data['Cluster'].sample(n=10)

781846    6
457855    4
473820    7
406938    3
406314    6
536231    6
171459    7
896236    1
675481    7
441098    2
Name: Cluster, dtype: int32

In [18]:
data['Cluster'].value_counts()

0    139956
7    137609
1    126161
4    122057
5    121403
6    120547
2    118847
3    113629
Name: Cluster, dtype: int64

In [19]:
data.head()

,UserID,MovieID,Rating,Drama,Animation,Musical,Action,Comedy,Adventure,Romance,...,Crime,Western,Documentary,Mystery,Horror,Sci-Fi,Film-Noir,War,Fantasy,Cluster
0,1.0,1193,5.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2.0,1193,5.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,12.0,1193,4.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,15.0,1193,4.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,17.0,1193,5.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [20]:
# When we merge the dataframe for a single movie multiple rows were created so a single movie is allotted
# to many clusters so here we allot a single cluster to a movie 
# the Cluster which occurs maximum number of times is alloted to the movie  
e = []
def fi(group):
    a = pd.DataFrame(group)
    b = pd.DataFrame(a['Cluster'].value_counts())
    d = a.index 
    c = [a['MovieID'][d[0]],int(b.idxmax())]
    e.append(c)
    

In [21]:
data.groupby("MovieID").apply(lambda x: fi(x))

""


In [22]:
e = pd.DataFrame(e)

In [23]:
e.head()

,0,1
0,1,7
1,1,7
2,2,7
3,3,2
4,4,5


In [24]:
# I Dont know why always the column name shift according to its will :(
# Here just the column names are swapped
e.rename(columns = {0:'MovieID',1:'Cluster'},inplace=True)
e.drop_duplicates(inplace=True)

In [25]:
e.head(10)

,MovieID,Cluster
0,1,7
2,2,7
3,3,2
4,4,5
5,5,5
6,6,7
7,7,1
8,8,2
9,9,1
10,10,7


In [26]:
data1 = pd.read_csv('movies data.csv')
new_data = pd.merge(e , data1 , how='outer', on='MovieID')

In [27]:
# These were the movies we deleted while merging the file  
new_data.isnull().sum()

MovieID         0
Cluster       177
Unnamed: 0      0
Title           0
Genres          0
dtype: int64

In [28]:
# We can delete the movies but I just label them randomly :)
new_data.fillna(random.randint(0,8),inplace=True)

In [29]:
new_data.isnull().sum()

MovieID       0
Cluster       0
Unnamed: 0    0
Title         0
Genres        0
dtype: int64

In [30]:
#This function select the cluster for a user according the the user choice
def select_c():
    global l
    print('Select The Movies Id you would like to watch:')
    l=[]
    for i in range(15):
        l.append(random.randint(0,3883))
    for i in l:
        print(new_data['MovieID'][i] , new_data['Title'][i],sep='--->')
    print('--------------------------------------------------------------------')
    l = int(input())
    l = new_data['Cluster'][new_data.MovieID == l]

In [31]:
# This is the main function which recommend you movies
def main():
    ans = False
    while not ans:
        select_c()
        print(new_data['Title'][new_data.Cluster == int(l)].sample(n=10))
        print('--------------------------------------------------------------------')
        print('Do you like these movies(y/n)')
        abc = input()
        while ((abc =='y') or (abc == 'Y')):          
            print(new_data['Title'][new_data.Cluster == int(l)].sample(n=10))
            print('--------------------------------------------------------------------')
            print('Want more!!!!(y/n)')
            abc = input()
            if ((abc =='N') or (abc == 'n')):
                ans =True

In [ ]:
main()

Select The Movies Id you would like to watch:
2349--->Mona Lisa (1986)
1490--->B*A*P*S (1997)
3025--->Rough Night in Jericho (1967)
2190--->Why Do Fools Fall In Love? (1998)
828--->Adventures of Pinocchio, The (1996)
1699--->Butcher Boy, The (1998)
982--->Picnic (1955)
3213--->Batman: Mask of the Phantasm (1993)
2551--->Dead Ringers (1988)
3403--->Raise the Titanic (1980)
2436--->Tea with Mussolini (1999)
1055--->Shadow Conspiracy (1997)
1163--->Mina Tannenbaum (1994)
2700--->South Park: Bigger, Longer and Uncut (1999)
1682--->Truman Show, The (1998)
--------------------------------------------------------------------
